In [2]:
pip install -U python-dotenv

  Using cached python_dotenv-0.19.0-py2.py3-none-any.whl (17 kB)
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install web3.py

ERROR: Could not find a version that satisfies the requirement web3.py (from versions: none)
ERROR: No matching distribution found for web3.py
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install --upgrade web3

  Using cached web3-5.21.0-py3-none-any.whl (482 kB)
  Using cached lru-dict-1.1.7.tar.gz (10 kB)
  Using cached protobuf-3.17.3-cp37-cp37m-macosx_10_9_x86_64.whl (1.0 MB)
  Using cached eth_account-0.5.5-py3-none-any.whl (100 kB)
  Using cached rlp-2.0.1-py2.py3-none-any.whl (20 kB)
  Using cached bitarray-1.2.2.tar.gz (48 kB)
  Using cached eth_rlp-0.2.1-py3-none-any.whl (5.0 kB)
  ERROR: Command errored out with exit status 1:
   command: /Users/macbookmaladocs/opt/anaconda3/envs/dev/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/mp/dkfg6s7x0l133jhmn47k7cp80000gn/T/pip-install-dv_p5ec_/lru-dict/setup.py'"'"'; __file__='"'"'/private/var/folders/mp/dkfg6s7x0l133jhmn47k7cp80000gn/T/pip-install-dv_p5ec_/lru-dict/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/mp/dkfg6s7x0l133jhmn47k

In [13]:
import subprocess
import json
from dotenv import load_dotenv
from web3 import web3
from bit import PrivateKeyTestnet
from web3 import middleware,Account
from web3.gas_strategies.time_based import medium_gas_price_strategy
from web3.middleware import geth_poa_middleware
from pprint import pprint
from constants import *

ModuleNotFoundError: No module named 'web3'

In [14]:
load_dotenv()

True

In [15]:
import os

In [16]:
mnemonic=os.getenv("mnemonic")

In [17]:
# Import constants.py and necessary functions from bit and web3
BTC = 'btc'
ETH = 'eth'
BTCTEST = 'btc-test'

In [18]:
# Create a function called `derive_wallets`
def derive_wallets(coin):
    command =  f'php ./derive -g --mnemonic="{mnemonic}" --cols=all --format=json --coin="{coin}" --numderive=3'
    #print (command)
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    #print(output)
    return json.loads(output)

In [19]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {
    'BTCTEST': derive_wallets(BTCTEST,3),
    'ETH': derive_wallets(ETH,3),
}

TypeError: derive_wallets() takes 1 positional argument but 2 were given

In [20]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    if coin ==ETH:
        return Account.PrivateKeyToAccount(priv_key)
    elif coin==BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [21]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin,account,to,amount):
    if coin == ETH:
        value = w3.toWei(amount, "ether") 
        gasEstimate = w3.eth.estimateGas({ "to": to, "from": account, "amount": value })
        return {
            "to": to,
            "from": account,
            "value": value,
            "gas": gasEstimate,
            "gasPrice": w3.eth.generateGasPrice(),
            "nonce": w3.eth.getTransactionCount(account),
            "chainId": w3.eth.chain_id
        }
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])


In [22]:

# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin,account,to,amount):
    if coin == BTCTEST:
        raw_tx = create_tx(coin,account,to,amount)
        signed = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)
    elif coin == ETH:
        raw_tx = create_tx(coin,account,to,amount)
        signed = account.signTransaction(raw_tx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
